In [ ]:
import arcpy
from arcpy.sa import *

arcpy.CheckOutExtension("Spatial")
arcpy.env.workspace = "T:\\Students\\chimam\\submissions\\Lab5\\SuitabilityModeling\\Data.gdb"
arcpy.env.overwriteOutput = True

# Prompt user to enter weights for slope, landuse, and distance to streams
slopeValue = float(input("Enter weight for slope (0-1): "))
landuseValue = float(input("Enter weight for landuse (0-1): "))
dist2strValue = float(input("Enter weight for distance to streams (0-1): "))

# Check out any necessary licenses.
arcpy.CheckOutExtension("3D")
#arcpy.CheckOutExtension("spatial")
arcpy.CheckOutExtension("ImageAnalyst")

# Set Geoprocessing environments
arcpy.env.snapRaster = "T:\\Students\\chimam\\submissions\\Lab5\\SuitabilityModeling\\Data.gdb\\Elevation"
arcpy.env.extent = "439952.113762345 200181.284694512 513122.113762345 253671.284694512"
arcpy.env.cellSize = "30"
arcpy.env.mask = "T:\\Students\\chimam\\submissions\\Lab5\\SuitabilityModeling\\Data.gdb\\Elevation"
Streams = "Streams"
Elevation = arcpy.Raster("Elevation")
LandUse = arcpy.Raster("LandUse")

# Process: Distance Accumulation (Distance Accumulation) (sa)
Distance_Streams = "T:\\Students\\chimam\\submissions\\Lab5\\SuitabilityModeling\\Output.gdb\\Distance_Streams"
Distance_Accumulation = Distance_Streams
backRaster = "T:\\Students\\chimam\\submissions\\Lab5\\SuitabilityModeling\\Output.gdb\\backRaster"
Out_source_direction_raster = ""
Out_source_location_raster = ""
Distance_Streams = arcpy.sa.DistanceAccumulation(Streams, "", "", "", "", "BINARY 1 -30 30", "", "BINARY 1 45", backRaster, Out_source_direction_raster, Out_source_location_raster, "", "", "", "", "PLANAR")
Distance_Streams.save(Distance_Accumulation)

backRaster = arcpy.Raster(backRaster)

# Process: Slope (Slope) (sa)
SlopeRaster = "T:\\Students\\chimam\\submissions\\Lab5\\SuitabilityModeling\\Output.gdb\\SlopeRaster"
Slope = SlopeRaster
SlopeRaster = arcpy.sa.Slope(Elevation, "PERCENT_RISE", 0.30480060960121924, "PLANAR", "Foot_US")
SlopeRaster.save(Slope)


# Process: Reclassify (Reclassify) (sa)
Reclass_Slope = "T:\\Students\\chimam\\submissions\\Lab5\\SuitabilityModeling\\Output.gdb\\Reclass_Slope"
Reclassify = Reclass_Slope
Reclass_Slope = arcpy.sa.Reclassify(SlopeRaster, "VALUE", "0 3 0;3 10 3;10 25 6;25 90 10", "DATA")
Reclass_Slope.save(Reclassify)


# Process: Reclassify (2) (Reclassify) (sa)
Reclass_Landuse = "T:\\Students\\chimam\\submissions\\Lab5\\SuitabilityModeling\\Output.gdb\\Reclass_Landuse"
Reclassify_2_ = Reclass_Landuse
Reclass_Landuse = arcpy.sa.Reclassify(LandUse, "VALUE", "1 1;2 1;3 1;4 2;5 8;6 10;7 10;8 10;9 6;10 2;11 10;12 1", "DATA")
Reclass_Landuse.save(Reclassify_2_)


# Process: Rescale by Function (Rescale by Function) (sa)
Rescale_Dist2Stream = "T:\\Students\\chimam\\submissions\\Lab5\\SuitabilityModeling\\Output.gdb\\Rescale_Dist2Stream"
Rescale_by_Function = Rescale_Dist2Stream
Rescale_Dist2Stream = arcpy.sa.RescaleByFunction(Distance_Streams, [["MSSMALL", "", "", "", "", 1, 1, ""]], 1, 10)
Rescale_Dist2Stream.save(Rescale_by_Function)


# Process: Raster Calculator (Raster Calculator) (sa)
SuitableSites = "T:\\Students\\chimam\\submissions\\Lab5\\SuitabilityModeling\\Output.gdb\\SuitableSites"
Raster_Calculator = SuitableSites
SuitableSites =  (Reclass_Slope * slopeValue) + (Reclass_Landuse * landuseValue) + ( Rescale_Dist2Stream * dist2strValue)
SuitableSites.save(Raster_Calculator)

print("Suitable sites raster created successfully")